In [1]:
# Dependencies
from gazpacho import Soup
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Run Chrome to assist with identifying another set of html calls.
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/normanadkins/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [3]:
# Visit the new URL for preview
url = "https://s3.amazonaws.com/tripdata/index.html"
browser.visit(url)

In [4]:
# Requests
html_page = requests.get(url)
soup = BeautifulSoup(html_page.content, "html.parser")

In [5]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Create an list of csv links
rows = []

# Loop through the links to append 
for link in soup.find_all('a'):
    
    # Append the zip "clean" links
    rows.append(link.get('href'))

# Only capture the zip files that are the focus.
links = rows[2:104]

# Print to confirm the changes
links

['https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201308-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201309-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201310-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201311-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201312-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201401-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201402-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201403-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201404-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201405-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201406-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201407-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201408-citibike-tripdata.zip',
 'https://s3.amazonaws.com/tripdata/201409-citibike-tripdata.z

In [6]:
# Close out browser session
browser.quit()

In [8]:
# Start a DataFrame
master_df = pd.DataFrame()

# Only do one at a time due to memory limitation
for link in links[0:3]:
    # open url
    resp = urlopen(link)
    # read zip
    zipfile = ZipFile(BytesIO(resp.read()))
    # Get csv file name
    fname = zipfile.namelist()[0]
    # Data Frame
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    # Close zip
    zipfile.close()
    # Concatenate DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
    # Free up memory
    del df
    # Print Confirmation
    print(f'{fname} has been completed')
    
# Save Data Frame for Tableau Visuals
master_df.to_csv("data/bike_rental.csv")

2013-07 - Citi Bike trip data.csv has been completed
2013-08 - Citi Bike trip data.csv has been completed
2013-09 - Citi Bike trip data.csv has been completed
2013-10 - Citi Bike trip data.csv has been completed
2013-11 - Citi Bike trip data.csv has been completed
2013-12 - Citi Bike trip data.csv has been completed
2014-01 - Citi Bike trip data.csv has been completed
2014-02 - Citi Bike trip data.csv has been completed
2014-03 - Citi Bike trip data.csv has been completed
2014-04 - Citi Bike trip data.csv has been completed
2014-05 - Citi Bike trip data.csv has been completed
2014-06 - Citi Bike trip data.csv has been completed
2014-07 - Citi Bike trip data.csv has been completed
2014-08 - Citi Bike trip data.csv has been completed
201409-citibike-tripdata.csv has been completed
201410-citibike-tripdata.csv has been completed
201411-citibike-tripdata.csv has been completed
201412-citibike-tripdata.csv has been completed
201501-citibike-tripdata.csv has been completed
201502-citibike-tr

In [ ]:
# Only do one at a time due to memory limitation
for link in links[26:30]:
    # open url
    resp = urlopen(link)
    # read zip
    zipfile = ZipFile(BytesIO(resp.read()))
    # Get csv file name
    fname = zipfile.namelist()[0]
    # Data Frame
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    # Close zip
    zipfile.close()
    # Concatenate DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
    # Free up memory
    del df
    # Print Confirmation
    print(f'{fname} has been completed')
    
# Save Data Frame for Tableau Visuals
master_df.to_csv("data/bike_rental.csv")

In [ ]:
for link in links[0:25]:
    # open url
    resp = urlopen(link)
    # read zip
    zipfile = ZipFile(BytesIO(resp.read()))
    # Get csv file name
    fname = zipfile.namelist()[0]
    # Data Frame
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    # Close zip
    zipfile.close()
    # Concatenate DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
    
    # Print Confirmation
    print(f'{fname} has been completed')
    
# Save Data Frame for Tableau Visuals
master_df.to_csv("data/bike_rental.csv")

In [ ]:
# Start a DataFrame
master_df = pd.DataFrame()

for link in links[26:50]:
    # open url
    resp = urlopen(link)
    # read zip
    zipfile = ZipFile(BytesIO(resp.read()))
    # Get csv file name
    fname = zipfile.namelist()[0]
    # Data Frame
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    # Close zip
    zipfile.close()
    # Concatenate DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
    
    # Print Confirmation
    print(f'{fname} has been completed')
    
# Save Data Frame for Tableau Visuals
master_df.to_csv("data/bike_rental.csv")

In [ ]:
# Start a DataFrame
master_df = pd.DataFrame()

for link in links[51:75]:
    # open url
    resp = urlopen(link)
    # read zip
    zipfile = ZipFile(BytesIO(resp.read()))
    # Get csv file name
    fname = zipfile.namelist()[0]
    # Data Frame
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    # Close zip
    zipfile.close()
    # Concatenate DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
    
    # Print Confirmation
    print(f'{fname} has been completed')
    
# Save Data Frame for Tableau Visuals
master_df.to_csv("data/bike_rental.csv")

In [ ]:
# Start a DataFrame
master_df = pd.DataFrame()

for link in links[76:102]:
    # open url
    resp = urlopen(link)
    # read zip
    zipfile = ZipFile(BytesIO(resp.read()))
    # Get csv file name
    fname = zipfile.namelist()[0]
    # Data Frame
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    # Close zip
    zipfile.close()
    # Concatenate DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
    
    # Print Confirmation
    print(f'{fname} has been completed')
    
# Save Data Frame for Tableau Visuals
master_df.to_csv("data/bike_rental.csv")